In [191]:
from google import genai
import os
from dotenv import load_dotenv
import duckdb
load_dotenv()
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

# response = client.models.generate_content(
#     model="gemini-2.0-flash", contents="Ge mig en historia om en kanin som bor i skogen på 3 meningar"
# )
# print(response.text)



In [192]:
with duckdb.connect("../../ads_data.duckdb") as con: 
    df_mart = con.execute("SELECT * FROM mart.mart_kultur_media").df()
df_test = df_mart #[df_mart["occupation_field"] == "Kultur, media, design"]
# df_test.to_csv("sample_ads.csv") # måste göra mer finjusteringar! kan inte läsa allas description direkt!


In [267]:
descriptions = df_test["description"]

# top_employer = df_test.groupby("occupation")[["vacancies", "description"]].sum().sort_values(by="vacancies", ascending=False).reset_index()
# top_employer = df_test.groupby("employer_name")[["vacancies", "description"]].sum().sort_values(by="vacancies", ascending=False).reset_index()
top_employer = df_test.groupby("workplace_city")[["vacancies", "description"]].sum().sort_values(by="vacancies", ascending=False).reset_index()

top_employer
# df_test

,workplace_city,vacancies,description
0,ej angiven,624,"Vi söker nu nya artister, musiker, skådespelar..."
1,Stockholm,166,Vi söker dig som vill bli spindeln i nätet på ...
2,Göteborg,138,Göteborgs universitet möter samhällets utmanin...
3,Malmö,102,"Ref: 20251128 \n\nHälsa-, vård- och omsorgsför..."
4,Boden,74,Research shows that women and other under-repr...
...,...,...,...
93,Örnsköldsvik,1,Känner du att du egentligen vill arbeta med nå...
94,Östersund,1,Vill du bli vår framtida Visuell Merchandiser?...
95,Österåker,1,Internationella Engelska Skolan (IES) is a lea...
96,Östra göinge,1,Östra Göinge kommun med drygt 14 000 invånare ...


In [268]:
top_employer = df_test.groupby("workplace_city")[["vacancies", "description"]].sum().sort_values(by="vacancies", ascending=False).reset_index()
# nr_1 = top_employer[top_employer["employer_name"] == 'Exakta Photo AB']
# nr_1 = top_employer.iloc[0]["description"]
# top_employer["description"][0]
nr_1 = top_employer[["workplace_city","description"]].iloc[2]
nr_1.to_csv("for_gemini.txt")
# nr_1.to_csv("for_gemini.csv")
nr_1

workplace_city                                             Göteborg
description       Göteborgs universitet möter samhällets utmanin...
Name: 2, dtype: object

In [195]:
# nr_1 = top_employer["employer_name"]
# nr_1["description"].to_csv("top_employer.csv")
# nr_1["description"]
nr_1

occupation                                                  Tolk
description    Vi söker utbildare inom japanska. \nUppdraget ...
Name: 0, dtype: object

In [269]:
# df_test.reset_index()

In [ ]:
with open("for_gemini.txt", "r", encoding='utf-8') as file: 
    ad_text = file.read() 
# ad_text = nr_1["description"]
# Plocka ut de 5 vanligaste kraven de 5 vanligaste meriterade och antal av dessa som efterfrågas i dessa jobbannonser, samt en sammanfattning av krav och meriter som du skriver ut:
# f"Vad är de vanligaste kraven och meriterna inom {choice_sort}?", 
# f"Vad är de vanligaste arbetsuppgifterna inom {choice_sort}?"
question = "Vad är de vanligaste arbetsuppgifterna inom Göteborg?"
promt = f"""Du är en rekryterare inom Media, Kultur, Design.
Svara på denna fråga: {question} Genom att läsa dessa annonser: 
{ad_text}

Output ska vara i detta formatet enbart: 

{{
    "sammanfattning": ["sammanfattning"]
    "krav": [erfarenhet1, erfarenhet2, ...]
    "antal krav": [summa erfarenhet1..]
    "meriterande": [meriterande1,meriterande2, ... ]
    "antal meriterande": [summa meriterande1..]
    
}}
"""
#om du inte får ihop samma antal balansera ut det som fattas med null, 0
# om du inte får ihop 5 så lägg in den sista som null, 0


In [308]:
response = client.models.generate_content(
    model="gemini-2.0-flash", contents=promt
)
print(response.text)

```json
{
    "sammanfattning": [
        "Vanliga arbetsuppgifter inom Media, Kultur och Design i Göteborg inkluderar biblioteksarbete (informationssökning, referenshantering, undervisning), översättning/textproduktion för e-handel, ljud- och ljusteknik, värdskap på museum, motion design och videoredigering, inredningsdesign, ledning av konstprojekt,  och agera som skådespelare."
    ],
    "biblioteksarbete": [
        "Service i informationsdisk",
        "Framtagning och uppsättning av böcker",
        "Utskrift, kopiering och IT-service",
        "Ämnesbevakning",
        "Urval och förvärv av tryckta och elektroniska monografier",
        "Litteratursökning",
        "Referenshantering",
        "Handledning och undervisning inom informationssökning",
        "Hantera utlämnandeärenden i e-arkiv och analoga arkiv",
        "Besvara förfrågningar från allmänhet, myndigheter, forskare och media",
        "Göra sekretessbedömningar av allmänna handlingar",
        "Stödja verksamhet

In [309]:
# print(repr(response.text))
import json

data = response.text
cleaned = data.strip("```json") #.replace("\n", "").strip().replace("  ", "")
cleaned
# data

'\n{\n    "sammanfattning": [\n        "Vanliga arbetsuppgifter inom Media, Kultur och Design i Göteborg inkluderar biblioteksarbete (informationssökning, referenshantering, undervisning), översättning/textproduktion för e-handel, ljud- och ljusteknik, värdskap på museum, motion design och videoredigering, inredningsdesign, ledning av konstprojekt,  och agera som skådespelare."\n    ],\n    "biblioteksarbete": [\n        "Service i informationsdisk",\n        "Framtagning och uppsättning av böcker",\n        "Utskrift, kopiering och IT-service",\n        "Ämnesbevakning",\n        "Urval och förvärv av tryckta och elektroniska monografier",\n        "Litteratursökning",\n        "Referenshantering",\n        "Handledning och undervisning inom informationssökning",\n        "Hantera utlämnandeärenden i e-arkiv och analoga arkiv",\n        "Besvara förfrågningar från allmänhet, myndigheter, forskare och media",\n        "Göra sekretessbedömningar av allmänna handlingar",\n        "Stödja

In [303]:
type(cleaned)
import pandas as pd
data = json.loads(cleaned)

df_krav = pd.DataFrame({
    "krav": data["krav"], 
    "antal krav": data["antal krav"]
})
df_krav
# df = pd.DataFrame(data)
# df
# df = pd.DataFrame(df)
# df

,krav,antal krav
0,Mycket goda skriv- och språkkunskaper (svenska...,12
1,"God dator- och systemvana (Officepaketet, kass...",8
2,Samarbetsförmåga,7
3,Relevant akademisk utbildning (biblioteks- och...,7
4,"Erfarenhet inom området (receptionsarbete, öve...",5


In [302]:
df_merit = pd.DataFrame({
    "meriterande": data["meriterande"],
    "antal meriterande": data["antal meriterande"]
})
df_merit

,meriterande,antal meriterande
0,Erfarenhet från universitets-/högskolebibliotek,2
1,Erfarenhet av Litium eller liknande,2
2,Erfarenhet inom specifika program (CAD Rhino 3...,2
3,"Kunskap inom relevanta områden (e-handel, öppe...",5
4,"Ytterligare språkkunskaper (tyska, minoritetss...",2


In [304]:
sammanfattning = data["sammanfattning"]
sammanfattning

['De vanligaste kraven är språkkunskaper (främst svenska, engelska, danska, finska eller norska), god datorvana, samarbetsförmåga, och relevant utbildning. Meriterande är erfarenhet inom specifika system (Litium, CAD, etc.), kunskap inom relevanta områden (e-handel, konsthistoria etc.), och ytterligare språkkunskaper. Många annonser betonar personlig lämplighet och initiativförmåga.']

In [232]:
import plotly.express as px
# px.pie(df, names=["krav", "meriterande"], values=["antal krav", "antal meriterande"])
px.bar(df, x="krav", y="antal krav")


In [202]:
px.bar(df, y="antal meriterande", x="meriterande")